In [44]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import csv


In [45]:
# Top 100 Anime of all time according to IMDB
url = "https://www.imdb.com/list/ls057577566/"
response = requests.get(url)
soup = BeautifulSoup(response.content,'html.parser')

In [46]:
# Create lists for data to be scraped into
name = []   # name of the anime
rating = []  # rating of the anime
aired_duration = [] # released year - end year
average_runtime = [] # average duration of each anime episode
genre = [] # genre of the anime
MPAA_rating = [] # MPAA rating
votes = []

In [47]:
# Use bs4 to place all 100 anime tv-shows into a list
anime_data = soup.find_all('div', attrs={'class':'lister-item mode-detail'})

In [48]:
anime_data

[<div class="lister-item mode-detail">
 <div class="lister-item-image ribbonize" data-tconst="tt0213338">
 <a href="/title/tt0213338/"> <img alt="Cowboy Bebop" class="loadlate" data-tconst="tt0213338" height="209" loadlate="https://m.media-amazon.com/images/M/MV5BZDEzMjlkNDgtZjhkNi00N2RlLWFhZTYtNDE5OTc2NjZkODJlXkEyXkFqcGdeQXVyNjAwNDUxODI@._V1_UY209_CR6,0,140,209_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="140"/>
 </a> </div>
 <div class="lister-item-content">
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">1.</span>
 <a href="/title/tt0213338/">Cowboy Bebop</a>
 <span class="lister-item-year text-muted unbold">(1998–1999)</span>
 </h3>
 <p class="text-muted text-small">
 <span class="certificate">TV-14</span>
 <span class="ghost">|</span>
 <span class="runtime">650 min</span>
 <span class="ghost">|</span>
 <span class="genre">
 Animation, Action, Adventure            </span>
 </p>
 <div class="ipl-rating-widge

In [49]:
# iterate through all 100 movies
for data in anime_data:
    # extract all the names of the animes
    name.append(data.h3.a.text)  

    # extract all the aired_duration of the animes
    aired_duration.append(data.find(string=re.compile(r'\(.*\)')))

    # extract all the genres of the animes
    genre.append(data.p.find('span', attrs={'class':'genre'}).text)

    # extract all the mpaa certificates of the animes
    mpaa_review = data.p.find('span',class_='certificate')
    if mpaa_review:
        MPAA_rating.append(mpaa_review.text) 
    else:
        MPAA_rating.append('None')

    # extract all the average run time of the animes
    runtime_span = data.p.find('span', class_='runtime')
    if runtime_span:
        average_runtime.append(runtime_span.text)
    else:
        average_runtime.append("None")

    # extract all the ratings of the animes
    rate = data.find('span',class_='ipl-rating-star__rating')
    if rate:
        rating.append(rate.text)
    else:
        rating.append('None')

    # extract all the votes of the animes
    vote = data.find('span', attrs={'name':'nv'})
    if vote:
        votes.append(vote.text)
    else:
        votes.append("None")


In [50]:
# Clean the genre array by stripping the whitespaces and newlines within each element
temp_genre = []
for x in genre:
    temp_genre.append(x.strip())
genre = temp_genre

In [51]:
with open("scraped_top_100_anime.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(["title", "rating", "aired_duration","average_runtime", "genre","MPAA_rating", "votes"])
    for title, score, duration, avg_time, genres, mpaa, vote in zip(name,rating,aired_duration,average_runtime,genre,MPAA_rating,votes):
       # print(title + " " + score+ " " + duration + " " + avg_time + " " + genres + " "  + mpaa + " " + vote)
        writer.writerow([title, score, duration, avg_time, genres, mpaa, vote])
